# QAOA 

In [ ]:
from qiskit_aer import AerSimulator
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit_ibm_runtime import QiskitRuntimeService, Session, EstimatorV2 as Estimator, SamplerV2 as Sampler
from scipy.optimize import minimize
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import rustworkx as rx
from rustworkx.visualization import mpl_draw as draw_graph
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator

## The problem: 

The goal of this problem is to partition the vertices $V$ of a graph $G={V,E}$ into two sets $S$ and $T$ such that the number of edges $E$ between $S$ and $T$ traversed by this cut is maximized.

**Note:** In this examples all edges have weight 1.

In [ ]:
n = 5 # Number of vertices/nodes

graph = rx.PyGraph()
graph.add_nodes_from(np.arange(0, n, 1))
edge_list = [(0, 1, 1.0), (0, 2, 1.0), (0, 4, 1.0), (1, 2, 1.0), (2, 3, 1.0), (3, 4, 1.0)]
graph.add_edges_from(edge_list)
draw_graph(graph, node_size=600, with_labels=True)

## Map optimization problem to Hamiltonian

In [ ]:
edge_0 = graph.edge_list()[0] # 0th edge
edge_0

In [ ]:
graph.get_edge_data(edge_0[0], edge_0[1]) # weight of 0th edge

In [ ]:
def build_max_cut_paulis(graph):
    """
    Convert the graph to Pauli list.
    """
    pauli_list = []
    for edge in list(graph.edge_list()):
        # Instantiate pauli identity operators
        # Your code goes here: 
        
        # Set Z operators for every pair or nodes that are connected via an edge
        # Your code goes here:

        weight = graph.get_edge_data(edge[0], edge[1])
        # Join in reverse order from list of strings to a single string
        pauli_list.append(("".join(paulis)[::-1], weight))

    hamiltonian = SparsePauliOp.from_list(pauli_list)

    return hamiltonian


hamiltonian = build_max_cut_paulis(graph)

print("Cost Function Hamiltonian:", hamiltonian)

## Create an ansatz suitable for the QAOA hamiltonian

In [ ]:
# Create a QAOA 'ansatz' circuit using the hamiltonian that was derived before as cost operator.
# Your code goes here:



In [ ]:
ansatz.decompose(reps=3).draw('mpl')

In [ ]:
print(f"Number of parameters in ansatz: {ansatz.num_parameters}")
print(f"Number of qubits in ansatz: {ansatz.num_qubits}")

## Define cost function

In [ ]:
def cost_func_estimator(params, ansatz, hamiltonian, estimator):
    # Definie the cost function of this QAOA problem using a primitive unified block (pub).
    # Note: the cost is represented by the expecation value of the hamiltonian w.r.t. to the
    # trail wave function (ansatz circuit) and a particular set of prameters.
    
    # Your code goes here:
    
    objective_func_vals.append(cost)

    return cost

## Optimize circuits for quantum hardware execution

In [ ]:
def transpile_problem_hardware(backend,circuit,hamiltonian):
    pm = generate_preset_pass_manager(optimization_level=3,
                                    backend=backend)
    # map the virtual circuit to an ISA on physical qubits
    isa_circuit = pm.run(circuit)
    # transform the observable defined on virtual qubits to
    # an observable defined on all physical qubits
    isa_hamiltonian = hamiltonian.apply_layout(isa_circuit.layout)
    return isa_circuit, isa_hamiltonian

In [ ]:
sim_backend = AerSimulator()
#service = QiskitRuntimeService(channel='ibm_quantum')
#real_backend = service.least_busy(operational=True, simulator=False)
#real_backend

In [ ]:
isa_ansatz, isa_hamiltonian = transpile_problem_hardware(sim_backend,ansatz,hamiltonian)

## Evaluate cost landscape (sim backend)

In [ ]:
objective_func_vals = [] 
n_samples = 50
beta = np.linspace(0, np.pi, n_samples)
gamma = np.linspace(0, 2*np.pi, n_samples)
cost_landscape = np.zeros((n_samples,n_samples))

with Session(backend=sim_backend) as session:
    estimator = Estimator(mode=session)
    for i, beta_i in enumerate(beta):
        for j, gamma_j in enumerate(gamma):
            objective_func_vals = [] 
            params_ij = [beta_i,gamma_j]
            cost_landscape[i,j] = cost_func_estimator(params_ij, isa_ansatz, isa_hamiltonian, estimator)
            

In [ ]:
#%matplotlib widget
# Plot the cost landscape
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
beta_v, gamma_v = np.meshgrid(beta,gamma)
surf = ax.plot_surface(beta_v, gamma_v, cost_landscape, cmap=cm.cool,alpha=0.5)
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

**Note:**
- The landscape is $\pi$ periodic, which is due to the periodic nature of the rotation gates in our ansatz.
- There are 4 minima within the parameter sweep $\beta \in [0,\pi], \gamma \in [0,\pi]$

## Minimize cost (real backend)

In [ ]:
objective_func_vals = [] 
init_params = np.random.uniform(0,np.pi,size=ansatz.num_parameters)
init_params

In [ ]:
#isa_ansatz, isa_hamiltonian = transpile_problem_hardware(real_backend,ansatz,hamiltonian)

In [ ]:
with Session(backend=sim_backend) as session:
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    #estimator.options.dynamical_decoupling.enable = True
    #estimator.options.dynamical_decoupling.sequence_type = "XY4"
    #estimator.options.twirling.enable_gates = True
    #estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(isa_ansatz, isa_hamiltonian, estimator),
        method="COBYLA",
        tol=1e-3,
    )
    print(result)

In [ ]:
plt.figure()
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

In [ ]:
#%matplotlib widget
# Plot the cost landscape
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
beta_v, gamma_v = np.meshgrid(beta,gamma)
surf = ax.plot_surface(beta_v, gamma_v, cost_landscape, cmap=cm.cool,alpha=0.5)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.scatter(result.x[0],result.x[1],result.fun,color='black',s=30)

plt.show()

## Sample solution from optimized circuit

In [ ]:
# Assign the parameters to the 'isa_ansatz' that where used to evaluate the smallest observable

# Your code goes here:


In [ ]:
sampler = Sampler(mode=sim_backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"

pub= (optimized_circuit, )
job = sampler.run([pub])

## Post process results

In [ ]:
counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_int = {key: val/shots for key, val in counts_int.items()}
final_distribution_bin = {key: val/shots for key, val in counts_bin.items()}

print(final_distribution_bin)
most_likely_bitstring = max(final_distribution_bin, key=final_distribution_bin.get)
most_likely_bitlist = [int(ele) for ele in [*most_likely_bitstring]][::-1]
print(f"Most likely bitstring: {most_likely_bitlist}")

In [ ]:
def plot_result(graph, x):
    plt.figure()
    colors = ["tab:grey" if i == 0 else "tab:purple" for i in x]
    pos, default_axes = rx.spring_layout(graph), plt.axes(frameon=True)
    rx.visualization.mpl_draw(graph, node_color=colors, node_size=100, alpha=0.8, pos=pos)


plot_result(graph, most_likely_bitlist)